# Validation of Experiment 7.2: MNAR Training

In [ ]:
import numpy as np
import tensorflow as tf

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
import sys
from pathlib import Path
SCRIPT_DIR = os.path.dirname(os.path.abspath("__init__.py"))
SRC_DIR = Path(SCRIPT_DIR).parent.parent.absolute()
print(SCRIPT_DIR)
print(SRC_DIR)
sys.path.append(os.path.dirname(SRC_DIR))
sys.path.append(os.path.dirname(str(SRC_DIR) + '/models'))

In [ ]:
from src.models.model2 import create_model
from src.data_loaders.loading import get_val_sets
from src.experiments_evaluation.validation_helpers import calc_temporal_errors, calc_total_errors, calc_spatial_errors, scale_t2m_back, scale_slp_back, reshape_for_modelling, get_median_pred_days

In [ ]:
EXP_NAME = "ex7.2_emc_mnar_training"

In [ ]:
F = 5
H = 32
W = 64
CH = 5  # t2m, msl, msk1, msk2, elev
BS = 4
PERCENTAGE = "99"
MISSING_MECHANISM = "mnar"

MODEL_PATH = str(SRC_DIR) + f"/experiments_evaluation/{EXP_NAME}/model_checkpoint/p{PERCENTAGE}/"
MODEL_PATH

In [ ]:
strategy = tf.distribute.MirroredStrategy()
with strategy.scope():
    model = create_model(f=F, h=H, w=W, ch=CH, bs=BS)
    model.compile(optimizer=tf.keras.optimizers.Adam(), run_eagerly=None)
    model.load_weights(MODEL_PATH)

In [ ]:
val_x, val_y = get_val_sets(variant='b', percentage=PERCENTAGE, include_elevation=True, missing_type=MISSING_MECHANISM)

In [ ]:
x = reshape_for_modelling(val_x, seq_shift_reshape=True)
y = reshape_for_modelling(val_y, seq_shift_reshape=False)
print(x.shape)
print(y.shape)

# Only need variables
x = x[:3644]
y = y[:3644, ..., :2]
print(x.shape, y.shape)

In [ ]:
pred = model.predict(x, batch_size=BS)

In [ ]:
pred = get_median_pred_days(pred)
pred.shape

## Error on Climatology

In [ ]:
total_error, t2m_error, msl_error = calc_total_errors(y, pred)
print(f"Total error: {total_error}")
print(f"t2m error: {t2m_error}")
print(f"msl error: {msl_error}")

In [ ]:
# Save prediction
np.save("prediction/predicted.npy", pred)

# Get temporal Errors
temp_total_err, temp_t2m_err, temp_msl_err = calc_temporal_errors(y, pred)
np.save(f"errors/temp_total_err_{PERCENTAGE}p.npy", temp_total_err)
np.save(f"errors/temp_t2m_err_{PERCENTAGE}p.npy", temp_t2m_err)
np.save(f"errors/temp_slp_err_{PERCENTAGE}p.npy", temp_msl_err)

# Get spatial errors
spat_total_err, spat_t2m_err, spat_msl_err = calc_spatial_errors(y, pred)
np.save(f"errors/spat_total_err_{PERCENTAGE}p.npy", spat_total_err)
np.save(f"errors/spat_t2m_err_{PERCENTAGE}p.npy", spat_t2m_err)
np.save(f"errors/spat_slp_err_{PERCENTAGE}p.npy", spat_msl_err)

In [ ]:
print(f"t2m scaled: {scale_t2m_back(t2m_error, for_error=True)}")
print(f"msl scaled: {scale_slp_back(msl_error, for_error=True)}")

## Error on anomaly

In [ ]:
from src.config import TA_MU, TA_SIGMA, SLP_MU, SLP_SIGMA

In [ ]:
unit_val_ymean = np.load("../../data_sets/ymean_sets/validation_ymean.npy")

normalized_gt = np.load("../../data_sets/variant_b/validation.npy")
normalized_gt = normalized_gt[:3644, 6:38, 2:66]

normalized_val_ymean = (unit_val_ymean - np.array([TA_MU, SLP_MU])) / np.array([TA_SIGMA, SLP_SIGMA])
normalized_val_ymean = normalized_val_ymean[:3644]

In [ ]:
normalized_pred = np.load("prediction/predicted.npy")
    
norm_pred_anom = normalized_pred - normalized_val_ymean
norm_gt_anom = normalized_gt - normalized_val_ymean
    
normalized_anomaly_mae = np.mean(np.abs(norm_pred_anom - norm_gt_anom)) 
print(f"{normalized_anomaly_mae}")

### RMSE

In [ ]:
def calc_rmse(gt, pred):
    squared_diffs = np.square(gt - pred)
    mean_squared_diff = np.mean(squared_diffs)
    return np.sqrt(mean_squared_diff)

calc_rmse(norm_gt_anom, norm_pred_anom)